In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.35 and you have 0.31 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
%idle_timeout 60

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Connections to be included:
dev-glue-redshift-connection
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 7ba7e67f-8ff6-44af-816d-9a20be7e3161
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session 7ba7e67f-8ff6-44af-816d-9a20be7e3161 to get into ready status...
Session 7ba7e67f-8ff6-44af-816d-9a20be7e3161 has been created




In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [6]:
from sstm_transformation.tsm_builder import TSMBuilder
from pyspark.sql.functions import input_file_name
import pyspark.sql.functions as F



In [7]:
#Read All dfs 

df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
#df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
#df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/")
#df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
#df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")
#df_project_vitals =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project_vitals.parquet")


In [ ]:
df_project_vitals = df_project_vitals.withColumn("input_file", input_file_name())


In [ ]:
df_project_vitals = df_project_vitals.withColumn('projectId', F.element_at(F.split(F.col('input_file'), '/'), -2))


In [8]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [ ]:
df_project_vitals.show()
#s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/10387917/project_vitals.parquet

In [9]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm.{}".format(table_name),
                            "dbtable": "tsm.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [ ]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    #print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

In [ ]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    print(result)
    upload_result(result)
    
people_master_transformation(df_contact)

In [ ]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

In [ ]:
#Peoples Transformation
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    
phases_transformation(df_projecttype)

In [ ]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    print(result)
    upload_result(result)
    
value_transformation(df_meds)

In [10]:
#Peoples Transformation
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    return result
    #print(result)
    #upload_result(result)
    
result = cases_transformation(df_project)

+--------------------+
|    Practice_Type_ID|
+--------------------+
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
+--------------------+
only showing top 20 rows

root
 |-- Truve_Org_ID: string (nullable = false)
 |-- Client_Org_ID: string (nullable = false)
 |-- Case_ID: integer (nullable = true)
 |-- Practice_Type_ID: integer (nullable = true)
 |-- Is_Archived: boolean (nullable = true)
 |-- Incident_Date: date (nullable = true)

+------------+-------------+--------+----------------+-----------+-------------+
|Truve_Org_ID|Client_Org_ID| Case_ID|Practice_Type_ID|Is_Archived|In

In [ ]:
#Peoples Transformation
def intake_transformation(df_intake):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake)
    return result 

result = intake_transformation(df_intake)
#result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


In [ ]:
#Peoples Transformation
def casesummary_transformation(df_casesummary, df_project_vitals, df_project):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary, df_project_vitals, df_project)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary, df_project_vitals, df_project)

In [11]:
processed_project_df = result["CMS_Cases"]
processed_project_df.printSchema()

root
 |-- Truve_Org_ID: integer (nullable = true)
 |-- Client_Org_ID: string (nullable = false)
 |-- Case_ID: integer (nullable = true)
 |-- Practice_Type_ID: integer (nullable = true)
 |-- Is_Archived: boolean (nullable = true)
 |-- Incident_Date: date (nullable = true)


In [12]:
#Peoples Transformation
def casetype_transformation(df_casesummary, processed_project_df):
    #Read Raw Data
    result = builder.build_casetypes(df_casesummary, processed_project_df)
    #print(result)
    #return result
    upload_result(result)
    
casetype_result = casetype_transformation(df_casesummary, processed_project_df)

[StructField(caseType,StringType,true), StructField(Practice_Type_ID,IntegerType,true), StructField(Truve_Org_ID,StringType,false), StructField(Client_Org_ID,StringType,false), StructField(Case_Type_ID,IntegerType,true), StructField(Case_Type_Name,StringType,true)]
Writing CMS_CaseTypes
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 1, "Case_Type_Name": "County/Municipal Liability - Ante Litem", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 2, "Case_Type_Name": "Diminished Value (PD)", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 3, "Case_Type_Name": "Dog Bite", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": nul

In [10]:
%stop_session

Stopping session: 7ba7e67f-8ff6-44af-816d-9a20be7e3161
Stopped session.


In [ ]:
df_casesummary.select("caseType").distinct().show(10)

In [ ]:
casetype_df = casetype_result["CMS_CaseTypes"]


In [ ]:
casetype_df.show()